In [1]:
import pandas as pd 
from glob import glob #檔名模式匹配(filename pattern matching)模組(Module)
import numpy as np 
import nltk #英語NLP套件
import re
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *

### 載入爬蟲文本資料 ->髒亂 還有 缺失。

In [2]:
#獲取所有JPY的新聞路徑
files = glob('./Newsdata/CurrencyNews_1*/JPY*.csv')
print(files)

['./Newsdata\\CurrencyNews_105\\JPY10501.csv', './Newsdata\\CurrencyNews_105\\JPY10502.csv', './Newsdata\\CurrencyNews_105\\JPY10503.csv', './Newsdata\\CurrencyNews_105\\JPY10504.csv', './Newsdata\\CurrencyNews_105\\JPY10505.csv', './Newsdata\\CurrencyNews_105\\JPY10506.csv', './Newsdata\\CurrencyNews_105\\JPY10507.csv', './Newsdata\\CurrencyNews_105\\JPY10508.csv', './Newsdata\\CurrencyNews_105\\JPY10509.csv', './Newsdata\\CurrencyNews_105\\JPY10510.csv', './Newsdata\\CurrencyNews_105\\JPY10511.csv', './Newsdata\\CurrencyNews_105\\JPY10512.csv', './Newsdata\\CurrencyNews_106\\JPY10601.csv', './Newsdata\\CurrencyNews_106\\JPY10602.csv', './Newsdata\\CurrencyNews_106\\JPY10603.csv', './Newsdata\\CurrencyNews_106\\JPY10604.csv', './Newsdata\\CurrencyNews_106\\JPY10605.csv', './Newsdata\\CurrencyNews_106\\JPY10606.csv', './Newsdata\\CurrencyNews_106\\JPY10607.csv', './Newsdata\\CurrencyNews_106\\JPY10608.csv', './Newsdata\\CurrencyNews_106\\JPY10609.csv', './Newsdata\\CurrencyNews_106\\JP

In [3]:
##載入資料(並選擇新聞本文欄位)

data = pd.DataFrame()

for file in files:
    #只要'date','body','source_title'這幾個欄位，編碼為ANSI
    df = pd.read_csv(file, usecols=['date','body','source_title'],
                    dtype={ 'date': str, 'body':str,'source_title':str},
                    encoding = "ANSI")
    #反轉index順序以符合時間
    df = df.reindex(index=df.index[::-1])
    #把所有df合併
    data = pd.concat([data,df],ignore_index=True)
#重設index
df_news = data.reset_index(drop=True)
df_news

,date,body,source_title
0,2016-01-01,2015 was a year of tremendous political and ec...,Forbes
1,2016-01-01,It's 2016! 4 trades for the New Year\r\n\r\nTh...,CNBC
2,2016-01-01,Airbus Group SE landed its first new airline c...,International Business Times
3,2016-01-02,Investing is becoming more of a grind. Expect ...,CNBC
4,2016-01-02,Mark Wilson/Getty ImagesThe world is neverthel...,Business Insider
...,...,...,...
34574,2020-12-31,TOKYO -- Some major Asian bourses were closed ...,MarketWatch
34575,2020-12-31,Nation's startup scene is beginning to show so...,The Wall Street Journal
34576,2020-12-31,The British pound surged Thursday to a 2.5-yea...,International Business Times
34577,2020-12-31,"Los Angeles, CA, Dec. 31, 2020 (GLOBE NEWSWIRE...",Yahoo Finance


In [4]:
# 刪除遺漏值
df_news = df_news.dropna()
df_news = df_news.reset_index(drop = True)

In [5]:
df_news.shape

(34579, 3)

### 處理時間

In [6]:
#確認時間是否正確 ->2021/5 的日期格式不對。
for i in df_news["date"]:
    print(i)

2016-01-01
2016-01-01
2016-01-01
2016-01-02
2016-01-02
2016-01-03
2016-01-03
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07
2016-01-07

2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-03-31
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-01
2016-04-02
2016-04-02
2016-04-02
2016-04-02
2016-04-03
2016-04-03
2016-04-03
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04
2016-04-04

2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-29
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-06-30
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-01
2016-07-02
2016-07-02
2016-07-02
2016-07-02
2016-07-02

2016-10-03
2016-10-03
2016-10-03
2016-10-03
2016-10-03
2016-10-03
2016-10-03
2016-10-03
2016-10-03
2016-10-03
2016-10-03
2016-10-03
2016-10-03
2016-10-03
2016-10-03
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-04
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-05
2016-10-06
2016-10-06
2016-10-06
2016-10-06
2016-10-06
2016-10-06
2016-10-06
2016-10-06
2016-10-06
2016-10-06
2016-10-06
2016-10-06
2016-10-06

2017-03-27
2017-03-27
2017-03-27
2017-03-27
2017-03-27
2017-03-27
2017-03-27
2017-03-27
2017-03-27
2017-03-27
2017-03-27
2017-03-27
2017-03-27
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-28
2017-03-29
2017-03-29
2017-03-29
2017-03-29
2017-03-29
2017-03-29
2017-03-29
2017-03-29
2017-03-29
2017-03-29
2017-03-29
2017-03-29
2017-03-29
2017-03-29
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-30
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-03-31
2017-04-01
2017-04-01
2017-04-01
2017-04-01
2017-04-01
2017-04-02
2017-04-02
2017-04-02

...

2019-01-30
2019-01-30
2019-01-30
2019-01-30
2019-01-30
2019-01-30
2019-01-30
2019-01-30
2019-01-30
2019-01-30
2019-01-30
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-01-31
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-01
2019-02-02
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-04
2019-02-05
2019-02-05
2019-02-05
2019-02-05
2019-02-05
2019-02-05

2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-12
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-13
2019-09-15
2019-09-15
2019-09-15
2019-09-15
2019-09-15
2019-09-15
2019-09-15
2019-09-15
2019-09-15
2019-09-15
2019-09-15
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16
2019-09-16

2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-21
2019-12-21
2019-12-21
2019-12-21
2019-12-21
2019-12-21
2019-12-21
2019-12-21
2019-12-21
2019-12-21
2019-12-21
2019-12-21
2019-12-21
2019-12-21
2019-12-22
2019-12-22
2019-12-22
2019-12-22
2019-12-22
2019-12-22
2019-12-22
2019-12-22
2019-12-22
2019-12-22
2019-12-22
2019-12-22
2019-12-22
2019-12-22
2019-12-22
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-23
2019-12-24
2019-12-24
2019-12-24
2019-12-24
2019-12-24

2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-15
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-16
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-17
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-18
2020-06-19
2020-06-19
2020-06-19

2020-11-27
2020-11-27
2020-11-27
2020-11-27
2020-11-27
2020-11-27
2020-11-27
2020-11-27
2020-11-28
2020-11-28
2020-11-28
2020-11-29
2020-11-29
2020-11-29
2020-11-29
2020-11-29
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-11-30
2020-12-01
2020-12-01
2020-12-01
2020-12-01
2020-12-01
2020-12-01
2020-12-01
2020-12-01
2020-12-01
2020-12-01
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-02
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-03
2020-12-04
2020-12-04
2020-12-04
2020-12-04
2020-12-04
2020-12-04
2020-12-04
2020-12-04
2020-12-04
2020-12-04
2020-12-04
2020-12-04
2020-12-06
2020-12-06
2020-12-06

### 把奇怪的日期處理掉

In [7]:
df_news = df_news[df_news["date"].str.contains("[0-9]*-[0-9]*-[09]*")]

### 處理新聞title

In [8]:
print(df_news.shape[0])
#瞭解個來源新聞分布數量
print(df_news.drop_duplicates(subset = "source_title"))

34579
             date                                               body  \
0      2016-01-01  2015 was a year of tremendous political and ec...   
1      2016-01-01  It's 2016! 4 trades for the New Year\r\n\r\nTh...   
2      2016-01-01  Airbus Group SE landed its first new airline c...   
4      2016-01-02  Mark Wilson/Getty ImagesThe world is neverthel...   
6      2016-01-03  Overseas investors cut holdings in 2015 even a...   
10     2016-01-04  Jan. 3, 2016 9:17 p.m. ET\r\n\r\nJapan's polic...   
17     2016-01-04  Japan is "no longer in a deflationary period",...   
22     2016-01-04  For the first since April 2014, all eurozone c...   
57     2016-01-06  The yen climbed against its rival currencies i...   
86     2016-01-07  LONDON (Reuters) - Global shares tumbled for a...   
204    2016-01-14  While many companies' shares are rising past t...   
28488  2020-01-01  Former Nissan (7201.T) boss Carlos Ghosn maste...   
28496  2020-01-02  Currency traders will not look back on 

In [14]:
dic = {} #資料有的tilte都加進去
news_resource = {} #只放有大於五篇以上的title
for i in df_news["source_title"]:
    if i not in dic:
        dic[i] = 1
    elif i in dic:
        dic[i] = dic[i]+1
        
for key, value in dic.items():
    if dic[key] > 5:
        news_resource[key] = value

In [15]:
#整理重複來源
news_resource

{'Forbes': 869,
 'CNBC': 5737,
 'International Business Times': 1846,
 'Business Insider': 1821,
 'Bloomberg Business': 5204,
 'The Wall Street Journal': 2550,
 'Financial Times': 2129,
 'Seeking Alpha': 5022,
 'MarketWatch': 3096,
 'Yahoo Finance': 5411,
 'The Motley Fool': 843,
 'Financial Times News': 50}

In [16]:
#這邊要人工判斷
news_resource['Yahoo Finance'] = news_resource['Yahoo Finance']
news_resource['Financial Times'] = news_resource['Financial Times']+news_resource['Financial Times News']
del news_resource['Financial Times News']

In [17]:
news_resource

{'Forbes': 869,
 'CNBC': 5737,
 'International Business Times': 1846,
 'Business Insider': 1821,
 'Bloomberg Business': 5204,
 'The Wall Street Journal': 2550,
 'Financial Times': 2179,
 'Seeking Alpha': 5022,
 'MarketWatch': 3096,
 'Yahoo Finance': 5411,
 'The Motley Fool': 843}

In [18]:
# 建立新聞來源list
new_publishment = []
for key,value in news_resource.items():
    new_publishment.append(key)
print(new_publishment)

['Forbes', 'CNBC', 'International Business Times', 'Business Insider', 'Bloomberg Business', 'The Wall Street Journal', 'Financial Times', 'Seeking Alpha', 'MarketWatch', 'Yahoo Finance', 'The Motley Fool']


In [19]:
np.save('./result/News_resource/JPY_new_resource.npy', news_resource)

#load method
# import numpy as np
# new_dict = np.load('./result/News_resource/EUR_new_resource.npy', allow_pickle='TRUE')
# print(new_dict)

In [20]:
#把df中的重複title替換掉，其餘的若不再新聞來源list則刪除
for i in range(df_news.shape[0]):
    if df_news["source_title"][i] == 'finance.yahoo.com':
        df_news["source_title"][i] = 'Yahoo Finance'
    elif df_news["source_title"][i] == 'Yahoo! Finance':
        df_news["source_title"][i] = 'Yahoo Finance'
    elif df_news["source_title"][i] == 'Financial Times News':
        df_news["source_title"][i] = 'Financial Times'
    elif df_news["source_title"][i] == 'Financial Times (FT)':
        df_news["source_title"][i] = 'Financial Times'
    elif df_news['source_title'][i] not in new_publishment:
        df_news.drop(i,inplace=True)



In [21]:
df_news = df_news.reset_index(drop =True)

In [22]:
df_news.shape

(34579, 3)

### 添加TF-IDF欄位

In [23]:
df_news.insert(2, "tfidftext", "") 
df_news.insert(2, "word_segmentation", "") 

df_news = df_news.reset_index(drop = True)
df_news

,date,body,word_segmentation,tfidftext,source_title
0,2016-01-01,2015 was a year of tremendous political and ec...,,,Forbes
1,2016-01-01,It's 2016! 4 trades for the New Year\r\n\r\nTh...,,,CNBC
2,2016-01-01,Airbus Group SE landed its first new airline c...,,,International Business Times
3,2016-01-02,Investing is becoming more of a grind. Expect ...,,,CNBC
4,2016-01-02,Mark Wilson/Getty ImagesThe world is neverthel...,,,Business Insider
...,...,...,...,...,...
34574,2020-12-31,TOKYO -- Some major Asian bourses were closed ...,,,MarketWatch
34575,2020-12-31,Nation's startup scene is beginning to show so...,,,The Wall Street Journal
34576,2020-12-31,The British pound surged Thursday to a 2.5-yea...,,,International Business Times
34577,2020-12-31,"Los Angeles, CA, Dec. 31, 2020 (GLOBE NEWSWIRE...",,,Yahoo Finance


In [24]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34579 entries, 0 to 34578
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date               34579 non-null  object
 1   body               34579 non-null  object
 2   word_segmentation  34579 non-null  object
 3   tfidftext          34579 non-null  object
 4   source_title       34579 non-null  object
dtypes: object(5)
memory usage: 1.3+ MB


In [25]:
#移除html標籤
def movehtml(text):
    dr = re.compile(r'<[^>]+>',re.S)
    return dr.sub('',text)

In [26]:
#移除文本標點以及數字
def removepunctuation(text):
    remove_chars = '[0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
    return re.sub(remove_chars, '', text)

In [27]:
#去除\n\r\t
def movebackslash(text):
    text = text.replace("\r", " ")
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    return text

In [28]:
#還原縮寫字
replacement_patterns = [
(r'won\'t', 'will not'),
(r'can\'t', 'cannot'),
(r'i\'m', 'i am'),
(r'ain\'t', 'is not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)n\'t', '\g<1> not'),
(r'(\w+)\'ve', '\g<1> have'),
(r'(\w+)\'s', '\g<1> is'),
(r'(\w+)\'re', '\g<1> are'),
(r'(\w+)\'d', '\g<1> would')]

class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            (s, count) = re.subn(pattern, repl, s)
        return s
        



In [29]:
#正規化處理
#正規化處理
replacer = RegexpReplacer()
for news_index in range(df_news.shape[0]):
    a = df_news["body"][news_index]
    b = ''.join(a) #確保a為str，而非list
    c = b.lower() #正規化，轉小寫
    result = replacer.replace(c) # 還原縮寫
    
    #斷句 ->bert用
    sentences = nltk.sent_tokenize(result)
    for i in range(len(sentences)):
        sentences[i] = removepunctuation(sentences[i]) #移除文本標點以及數字
        sentences[i] = movebackslash(sentences[i]) #去除\n\r\t
    df_news["word_segmentation"][news_index] = sentences
    
    #斷詞 -> word2vec
    result = removepunctuation(result) #移除文本標點以及數字
    result = movehtml(result) #移除html標籤
    result = movebackslash(result) #去除\n\r\t
    df_news["body"][news_index] = result
    

In [39]:
#Word Segmentation (斷詞)
for newsbody_index in range(df_news.shape[0]):
    a = df_news['body'][newsbody_index]
    newstokens = nltk.tokenize.word_tokenize(a)#將單篇文章的各句子斷詞
    df_news['tfidftext'][newsbody_index] = newstokens #將單篇文章的各句斷詞彙總到新串列，for loop 將該月所有文章集成到新串列
    

In [40]:
df_news

,date,body,word_segmentation,tfidftext,source_title
0,2016-01-01,was a year of tremendous political and econom...,[ was a year of tremendous political and econo...,"[was, a, year, of, tremendous, political, and,...",Forbes
1,2016-01-01,it is trades for the new year the fast money t...,"[it is , trades for the new year the fast mon...","[it, is, trades, for, the, new, year, the, fas...",CNBC
2,2016-01-01,airbus group se landed its first new airline c...,[airbus group se landed its first new airline ...,"[airbus, group, se, landed, its, first, new, a...",International Business Times
3,2016-01-02,investing is becoming more of a grind expect i...,"[investing is becoming more of a grind, expect...","[investing, is, becoming, more, of, a, grind, ...",CNBC
4,2016-01-02,mark wilsongetty imagesthe world is neverthele...,[mark wilsongetty imagesthe world is neverthel...,"[mark, wilsongetty, imagesthe, world, is, neve...",Business Insider
...,...,...,...,...,...
34574,2020-12-31,tokyo some major asian bourses were closed for...,[tokyo some major asian bourses were closed fo...,"[tokyo, some, major, asian, bourses, were, clo...",MarketWatch
34575,2020-12-31,nation is startup scene is beginning to show s...,[nation is startup scene is beginning to show ...,"[nation, is, startup, scene, is, beginning, to...",The Wall Street Journal
34576,2020-12-31,the british pound surged thursday to a year do...,[the british pound surged thursday to a year d...,"[the, british, pound, surged, thursday, to, a,...",International Business Times
34577,2020-12-31,los angeles ca dec globe newswire draganfly in...,[los angeles ca dec globe newswire draganfly i...,"[los, angeles, ca, dec, globe, newswire, draga...",Yahoo Finance


In [41]:
#詞性還原
for newsbody_index in range(df_news.shape[0]):
    pos = nltk.pos_tag(df_news['tfidftext'][newsbody_index])
    df_news['tfidftext'][newsbody_index] = pos

In [42]:
df_news['tfidftext'][0]

[('was', 'VBD'),
 ('a', 'DT'),
 ('year', 'NN'),
 ('of', 'IN'),
 ('tremendous', 'JJ'),
 ('political', 'JJ'),
 ('and', 'CC'),
 ('economic', 'JJ'),
 ('volatility', 'NN'),
 ('including', 'VBG'),
 ('continued', 'JJ'),
 ('stale', 'JJ'),
 ('mate', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('greek', 'JJ'),
 ('debt', 'NN'),
 ('crisis', 'NN'),
 ('reforms', 'VBZ'),
 ('the', 'DT'),
 ('greek', 'JJ'),
 ('impact', 'NN'),
 ('on', 'IN'),
 ('stock', 'NN'),
 ('markets', 'NNS'),
 ('the', 'DT'),
 ('chinese', 'JJ'),
 ('stock', 'NN'),
 ('market', 'NN'),
 ('summer', 'NN'),
 ('crash', 'NN'),
 ('plummeting', 'NN'),
 ('oil', 'NN'),
 ('prices', 'NNS'),
 ('do', 'VBP'),
 ('oil', 'NN'),
 ('prices', 'NNS'),
 ('foreshadow', 'VBD'),
 ('a', 'DT'),
 ('slump', 'NN'),
 ('in', 'IN'),
 ('us', 'PRP'),
 ('stock', 'NN'),
 ('markets', 'NNS'),
 ('and', 'CC'),
 ('repeated', 'VBD'),
 ('terrorists', 'NNS'),
 ('attacks', 'NNS'),
 ('from', 'IN'),
 ('paris', 'NN'),
 ('to', 'TO'),
 ('nigeria', 'NNS'),
 ('and', 'CC'),
 ('lebanon', 'NN'),
 (

In [43]:
#Lemmatization (字型還原)

for newsbody_index in range(df_news.shape[0]):
    wordnet_pos = []
    p = df_news['tfidftext'][newsbody_index]
    for word, tag in p:
        if tag.startswith('J'):
            wordnet_pos.append(nltk.corpus.wordnet.ADJ)
        elif tag.startswith('V'):
            wordnet_pos.append(nltk.corpus.wordnet.VERB)
        elif tag.startswith('N'):
            wordnet_pos.append(nltk.corpus.wordnet.NOUN)
        elif tag.startswith('R'):
            wordnet_pos.append(nltk.corpus.wordnet.ADV)
        else:
            wordnet_pos.append(nltk.corpus.wordnet.NOUN)
            
    # Lemmatizer
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    df_news['tfidftext'][newsbody_index] = [lemmatizer.lemmatize(p[n][0], pos=wordnet_pos[n]) for n in range(len(p))]
    

In [44]:
df_news['tfidftext'][0]

['be',
 'a',
 'year',
 'of',
 'tremendous',
 'political',
 'and',
 'economic',
 'volatility',
 'include',
 'continued',
 'stale',
 'mate',
 'of',
 'the',
 'greek',
 'debt',
 'crisis',
 'reform',
 'the',
 'greek',
 'impact',
 'on',
 'stock',
 'market',
 'the',
 'chinese',
 'stock',
 'market',
 'summer',
 'crash',
 'plummeting',
 'oil',
 'price',
 'do',
 'oil',
 'price',
 'foreshadow',
 'a',
 'slump',
 'in',
 'u',
 'stock',
 'market',
 'and',
 'repeat',
 'terrorist',
 'attack',
 'from',
 'paris',
 'to',
 'nigeria',
 'and',
 'lebanon',
 'yet',
 'u',
 'stock',
 'market',
 'largely',
 'weather',
 'the',
 'volatility',
 'the',
 'sp',
 'end',
 'the',
 'year',
 'roughly',
 'flat',
 'within',
 'this',
 'event',
 'have',
 'happen',
 'on',
 'only',
 'occasion',
 'since',
 'the',
 'nasdaq',
 'yet',
 'again',
 'outperform',
 'the',
 'sp',
 'which',
 'have',
 'be',
 'the',
 'case',
 'every',
 'year',
 'since',
 'the',
 'financial',
 'crisis',
 'and',
 'close',
 'the',
 'year',
 'at',
 'on',
 'the',


In [45]:
#remove Stopword (停用詞) 
nltk_stopwords = nltk.corpus.stopwords.words("english")

for newsbody_index in range(df_news.shape[0]):
    p = df_news['tfidftext'][newsbody_index]
    tokens = [token for token in p if token not in nltk_stopwords]
    df_news['tfidftext'][newsbody_index] = tokens

#現在即可得到所有詞幹還原的單詞列表 type(tokens) 為一維list。

#查驗用
#for token in tokens:
    #print(token)

In [46]:
len(df_news['tfidftext'][12])

263

In [43]:
print(df_news.shape)

body_length = 0
tfidftext_length = 0
body_length_list = []
tfidftext_length_list = []
for i in range(df_news.shape[0]):
    
    a = df_news['body'][i]
    newstokens = nltk.tokenize.word_tokenize(a)
    body_length += len(newstokens)
    body_length_list.append(len(newstokens))
    
    tfidftext_length += len(df_news['tfidftext'][i])
    tfidftext_length_list.append(len(df_news['tfidftext'][i]))
    
print("平均文本長度為：",body_length//df_news.shape[0])    
print("去除停用詞之平均文本長度為：",tfidftext_length//df_news.shape[0])

(35761, 4)
平均文本長度為： 813
去除停用詞之平均文本長度為： 478


## 至此為止，單篇文章的處理完成!

In [47]:
#建立一個新的dataframe來處理資料吧! -->還缺匯率漲跌的欄位後續再補。
date = []
word2vec_body = []
tfidf_body = []
bert_body = []
source_title = []

for newsbody_index in range(df_news.shape[0]):
    b = df_news['body'][newsbody_index]
    d = df_news['date'][newsbody_index]
    q = df_news['tfidftext'][newsbody_index]
    t = df_news['source_title'][newsbody_index]
    s = df_news['word_segmentation'][newsbody_index]
    word2vec_body.append(b)
    date.append(d)
    tfidf_body.append(" ".join(q))
    source_title.append(t)
    bert_body.append(s)
    
dic = {
    "date" : date,
    "word2vec_body" : word2vec_body,
    "tfidf_body" : tfidf_body,
    "bert_body" : bert_body,
    "source_title" : source_title
}

df = pd.DataFrame(dic)
df

,date,word2vec_body,tfidf_body,bert_body,source_title
0,2016-01-01,was a year of tremendous political and econom...,year tremendous political economic volatility ...,[ was a year of tremendous political and econo...,Forbes
1,2016-01-01,it is trades for the new year the fast money t...,trade new year fast money trader give trade ma...,"[it is , trades for the new year the fast mon...",CNBC
2,2016-01-01,airbus group se landed its first new airline c...,airbus group se land first new airline custome...,[airbus group se landed its first new airline ...,International Business Times
3,2016-01-02,investing is becoming more of a grind expect i...,investing become grind expect stay way analyst...,"[investing is becoming more of a grind, expect...",CNBC
4,2016-01-02,mark wilsongetty imagesthe world is neverthele...,mark wilsongetty imagesthe world nevertheless ...,[mark wilsongetty imagesthe world is neverthel...,Business Insider
...,...,...,...,...,...
34574,2020-12-31,tokyo some major asian bourses were closed for...,tokyo major asian bourse close new year holida...,[tokyo some major asian bourses were closed fo...,MarketWatch
34575,2020-12-31,nation is startup scene is beginning to show s...,nation startup scene begin show silicon valley...,[nation is startup scene is beginning to show ...,The Wall Street Journal
34576,2020-12-31,the british pound surged thursday to a year do...,british pound surge thursday year dollar peak ...,[the british pound surged thursday to a year d...,International Business Times
34577,2020-12-31,los angeles ca dec globe newswire draganfly in...,los angeles ca dec globe newswire draganfly in...,[los angeles ca dec globe newswire draganfly i...,Yahoo Finance


In [48]:
# 儲存一下資料!!!
df.to_csv("./Pre_data/news/JPYNews105109.csv")